# Dealing with Null Values

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("NY_Listings.csv")
len(df)

C:\Users\ericl\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,19,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


75749

75,749 rows leaves us plenty of wiggle room. Down below, you'll find a summary of null values and my tentative plan for dealing with them.

In [3]:
df.isnull().sum()

Listing ID                         0
Name                              31
Host ID                            0
Host Name                        232
Host Response Rate             45118
Host Is Superhost                  0
Host total listings count        232
Street                         31439
City                               0
Neighbourhood cleansed             0
State                              0
Country                            0
latitude                           0
longitude                          0
Property type                      0
Room type                          0
Accommodates                   31439
Bathrooms                      31581
Bedrooms                       31439
Amenities                      31721
Price                              0
Minimum nights                     0
Maximum nights                 31439
Availability 365                   0
Calendar last scraped          31439
Number of reviews                  0
Last Review Date               16683
R

### Removing unwanted columns

As you can see above, there's a concerning amount of null values. Some of these columns don't really matter. These include Name, Host Name, Calendar last scraped, and Last Review Date. Some of these columns could provide some information, but overall, I'm okay to exclude them from any work we do - especially since the text columns won't give us anything unless we do some language processing.

So for now, I'm just going to remove those columns from the dataset and see how things look after.

In [4]:
columnsToRemove = ["Name", "Host Name", "Calendar last scraped", "Last Review Date"]

df2 = df.drop(columns = columnsToRemove, axis=1)

df2.isnull().sum()

Listing ID                         0
Host ID                            0
Host Response Rate             45118
Host Is Superhost                  0
Host total listings count        232
Street                         31439
City                               0
Neighbourhood cleansed             0
State                              0
Country                            0
latitude                           0
longitude                          0
Property type                      0
Room type                          0
Accommodates                   31439
Bathrooms                      31581
Bedrooms                       31439
Amenities                      31721
Price                              0
Minimum nights                     0
Maximum nights                 31439
Availability 365                   0
Number of reviews                  0
Review Scores Rating               0
Review Scores Accuracy             0
Review Scores Cleanliness          0
Review Scores Checkin              0
R

### Removing rows with multiple columns missing

Here, we've narrowed down the nulls a good amount. Now, lets try removing the rows where Street is null. I have a suspicion that all the rows with 31,439 nulls are the same. We could try to impute these values, but I fear that columns like Accommodates and Bedrooms are too important to guess on. And this would still leave us with 44,310 rows.

In [5]:
df3 = df2[df2['Street'].notnull()]

df3.isnull().sum()

Listing ID                         0
Host ID                            0
Host Response Rate             13679
Host Is Superhost                  0
Host total listings count        232
Street                             0
City                               0
Neighbourhood cleansed             0
State                              0
Country                            0
latitude                           0
longitude                          0
Property type                      0
Room type                          0
Accommodates                       0
Bathrooms                        142
Bedrooms                           0
Amenities                        282
Price                              0
Minimum nights                     0
Maximum nights                     0
Availability 365                   0
Number of reviews                  0
Review Scores Rating               0
Review Scores Accuracy             0
Review Scores Cleanliness          0
Review Scores Checkin              0
R

In [6]:
len(df3)

44310

### Last four columns
Now that we're down to just four columns to look at, I'll take them one-by-one.

#### Bathrooms

For Bathrooms, these 142 columns only represent .3% of the dataset. I think if we partition out by room type and property type, we can imput the means.

In [7]:
bathroomsIncluded = df3[df3['Bathrooms'].notnull()]
bathroomsMissing = df3[df3['Bathrooms'].isna()]

df4 = df3[["Property type", "Room type", "Bathrooms"]]
df4 = df4[(df4["Property type"].isin(bathroomsMissing["Property type"].unique()))]
df4.groupby(by=["Property type", "Room type"]).mean()

Bathrooms
Property type   Room type                 
Apartment       Entire home/apt   1.099828
                Private room      1.096143
                Shared room       1.082711
Bed & Breakfast Entire home/apt   1.187500
                Private room      1.182692
                Shared room       1.000000
House           Entire home/apt   1.646916
                Private room      1.195214
                Shared room       1.692708

In [8]:
# For these values, I feel comfortable assigning 1 to all groups except for House:Entire and House:Shared.
# Those two groups will have 2 imputed.

bathroomsOne = bathroomsMissing[(bathroomsMissing['Property type'].isin(['Apartment', 'Bed & Breakfast'])) | 
                                (bathroomsMissing['Property type'] == 'House') &
                                (bathroomsMissing['Room type'] == 'Private room')]

bathroomsTwo = bathroomsMissing[(bathroomsMissing['Property type'] == 'House') &
                               (bathroomsMissing['Room type'] != 'Private room')]


bathroomsOne.loc[:, "Bathrooms"] = 1
bathroomsTwo.loc[:, "Bathrooms"] = 2

df5 = pd.concat([bathroomsIncluded, bathroomsOne, bathroomsTwo], ignore_index=True, axis=0)

df5.isnull().sum()

C:\Users\ericl\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Listing ID                         0
Host ID                            0
Host Response Rate             13679
Host Is Superhost                  0
Host total listings count        232
Street                             0
City                               0
Neighbourhood cleansed             0
State                              0
Country                            0
latitude                           0
longitude                          0
Property type                      0
Room type                          0
Accommodates                       0
Bathrooms                          0
Bedrooms                           0
Amenities                        282
Price                              0
Minimum nights                     0
Maximum nights                     0
Availability 365                   0
Number of reviews                  0
Review Scores Rating               0
Review Scores Accuracy             0
Review Scores Cleanliness          0
Review Scores Checkin              0
R

#### Amenities

Now onto Amenities. Since this is a list of values, I really am not certain we'll be able to impute this unless we build some association model that picks up common sets of amenities. Personally, I don't really want to do that right now. I'm going to try and just remove then, considering this is only .05% of the data.

In [9]:
df6 = df5[df5['Amenities'].notnull()]
df6.isnull().sum()

Listing ID                         0
Host ID                            0
Host Response Rate             13504
Host Is Superhost                  0
Host total listings count        227
Street                             0
City                               0
Neighbourhood cleansed             0
State                              0
Country                            0
latitude                           0
longitude                          0
Property type                      0
Room type                          0
Accommodates                       0
Bathrooms                          0
Bedrooms                           0
Amenities                          0
Price                              0
Minimum nights                     0
Maximum nights                     0
Availability 365                   0
Number of reviews                  0
Review Scores Rating               0
Review Scores Accuracy             0
Review Scores Cleanliness          0
Review Scores Checkin              0
R

#### Host Response Rate

I think this column is fairly important to predicting rating / price. Therefore, we can't just remove these rows. First, we can check for an overlap of hosts with included and missing values. For those that don't match - we may just need to impute the median.

In [10]:
responseIncluded = df6[df6['Host Response Rate'].notnull()]
responseMissing = df6[df6['Host Response Rate'].isna()]

responseIncluded_small = responseIncluded[["Host ID", "Host Response Rate"]]
responseMissing_small = responseMissing[["Host ID", "Host Response Rate"]]

responseMatches = pd.merge(left = responseIncluded_small, right = responseMissing_small, on="Host ID", how="inner")

responseMatches

,Host ID,Host Response Rate_x,Host Response Rate_y


In [11]:
responseMissing.loc[:, "Host Response Rate"] = responseIncluded["Host Response Rate"].median()

df7 = pd.concat([responseIncluded, responseMissing], ignore_index=True, axis=0)

df7.isnull().sum()

C:\Users\ericl\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Listing ID                       0
Host ID                          0
Host Response Rate               0
Host Is Superhost                0
Host total listings count      227
Street                           0
City                             0
Neighbourhood cleansed           0
State                            0
Country                          0
latitude                         0
longitude                        0
Property type                    0
Room type                        0
Accommodates                     0
Bathrooms                        0
Bedrooms                         0
Amenities                        0
Price                            0
Minimum nights                   0
Maximum nights                   0
Availability 365                 0
Number of reviews                0
Review Scores Rating             0
Review Scores Accuracy           0
Review Scores Cleanliness        0
Review Scores Checkin            0
Review Scores Communication      0
Review Scores Locati

#### Host total listings count

We can again check for overlap.

In [12]:
listingsIncluded = df7[df7['Host total listings count'].notnull()]
listingsMissing = df7[df7['Host total listings count'].isna()]

listingsIncluded_small = listingsIncluded[["Host ID", "Host total listings count"]]
listingsMissing_small = listingsMissing[["Host ID", "Host total listings count"]]

listingsMatches = pd.merge(left = listingsIncluded_small, right = listingsMissing_small, on="Host ID", how="inner")

listingsMatches

,Host ID,Host total listings count_x,Host total listings count_y


In [13]:
listingsMissing.loc[:, "Host total listings count"] = listingsIncluded["Host total listings count"].median()

df8 = pd.concat([listingsIncluded, listingsMissing], ignore_index=True, axis=0)

df8.isnull().sum()

C:\Users\ericl\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Listing ID                     0
Host ID                        0
Host Response Rate             0
Host Is Superhost              0
Host total listings count      0
Street                         0
City                           0
Neighbourhood cleansed         0
State                          0
Country                        0
latitude                       0
longitude                      0
Property type                  0
Room type                      0
Accommodates                   0
Bathrooms                      0
Bedrooms                       0
Amenities                      0
Price                          0
Minimum nights                 0
Maximum nights                 0
Availability 365               0
Number of reviews              0
Review Scores Rating           0
Review Scores Accuracy         0
Review Scores Cleanliness      0
Review Scores Checkin          0
Review Scores Communication    0
Review Scores Location         0
Review Scores Value            0
Reviews pe

In [54]:
len(df8)

44028

In [57]:
df8.to_csv(#r"C:\Users\ericl\OneDrive\Desktop\SMU\NY_Listings_Filtered.csv")

Now, we have no null values (can't be certain if there are values formatted incorrectly in the data though). I don't love some of these methods, so please let me know if y'all have any ideas! The row number is still about 50% more than the minimum, so I'm fairly happy with that.

By the way, I did check the LA Listings. This set has about 20,000 less rows to begin with, but a similar amount of null values.